<a href="https://colab.research.google.com/github/NikStork/nlp-emotions_classification/blob/master/nlp_emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP модель
Языковая модель для классификации эмоций, выражаемых в текстовых сообщениях

---

**Тема работы**: *Классификация эмоций, выражаемых в текстовых сообщениях, связанных с дорожным движением*



---



## **1. Импортируем необходимые библиотеки**






In [ ]:
%pip install pymystem3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
import re
from pymystem3 import Mystem
from nltk.corpus import stopwords
import nltk
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay

## **2. Подготовка данных**

* Загрузим датасет




In [ ]:
# Датасет №1: ru_sentiment_dataset
dfOne_train = pd.read_csv('/content/datasets/ru_sentiment_dataset/train.csv')
dfOne_test = pd.read_csv('/content/datasets/ru_sentiment_dataset/valid.csv')

# Датасет №2: twitter-dataset
# dfTwo_pos = pd.read_csv('/content/datasets/twitter-dataset/positive.csv',header=None,sep=';')
# dfTwo_neg = pd.read_csv('/content/datasets/twitter-dataset/negative.csv',header=None,sep=';')



* Изменим нумерацию классов в первом датасете



In [ ]:
tone_map = {  # Словарь для замены значений классов
    0: 0, 
    1: 1, 
    2: -1
    }  

dfOne_train['nsentiment'] = dfOne_train['sentiment'].map(tone_map)
dfOne_test['nsentiment'] = dfOne_test['sentiment'].map(tone_map)


* Объединим датасеты

In [ ]:
df = pd.concat([
    dfOne_train[['text','nsentiment']],
    dfOne_test[['text','nsentiment']],
    # pd.DataFrame({ 'text' : dfTwo_pos[3], 'nsentiment' : 1 }),
    # pd.DataFrame({ 'text' : dfTwo_neg[3], 'nsentiment' : -1 })
])

,text,nsentiment
0,".с.,и спросил его: о Посланник Аллаха!Ты пори...",1
1,Роднее всех родных Попала я в ГКБ №8 еще в дек...,1
2,Непорядочное отношение к своим работникам Рабо...,-1
3,"). Отсутствуют нормативы, Госты и прочее, что ...",1
4,У меня машина в руках 5 лет и это п...,1
...,...,...
21093,Несколько лет назад муж останавливался в этом ...,1
21094,Спасли от боли После родов у меня появились бо...,1
21095,з ролика понятно одно - девушка- наблюдатель ...,0
21096,"Остались всем довольны, дружелюбный персонал, ...",0


* Выполним лемматизацию

In [ ]:
regex = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
nltk.download('stopwords')
stopwords_ru = stopwords.words("russian")
mystem = Mystem()

def lemmatize(text, return_list=False):
    cleaned_text = re.sub(regex, ' ', text)
    tokens = []
    lemmas = mystem.lemmatize(cleaned_text)

    for lemma in lemmas:
        lemma = lemma.strip()
        if lemma and lemma not in stopwords_ru and lemma[0] != '@' and not lemma.isascii():
            tokens.append(lemma)

    if len(tokens) > 2:
        if return_list:
            return tokens
        else:
            return ' '.join(tokens)
    return None

# Выполним лемматизацию для всего датасета
tqdm.pandas()
df['ntext'] = df['text'].progress_apply(lemmatize)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  0%|          | 0/210989 [00:00<?, ?it/s]

In [ ]:
df.to_csv('lemmatized.csv.gz',encoding='utf-8',compression='gzip')

* Сохраним датасет, прошедший лемматизацию

In [ ]:
dataset = pd.read_csv('lemmatized.csv.gz',compression='gzip',encoding='utf-8')

* Разобьем датасет на обучающую и тестовую выборку, а также удалим пустые значения


In [ ]:
dfnn = df[~df['ntext'].isna()]
x_train,x_test,y_train,y_test = train_test_split(dfnn['ntext'],dfnn['nsentiment'],shuffle=True,stratify=dfnn['nsentiment'],test_size=0.1)

## **3. Обучение модели**

* Теперь подготовим модель и определим ее архитектуру. Мы используем многослойный двунаправленный LSTM RNN для нашей задачи. 

